Carlos Garcia - 21000475

### K-Folds Cross Validation

**K-fold.** Es una técnica de validación cruzada, cuyo objetivo es reducir el "bias" que puede presentarse al momento de entrenar un modelo de machine learning. Esta técnica se basa en el principio de evaluar el modelo sobre data que no ha sido utilizada para su entrenamiento, particionando el set de datos de entrenamiento para que este pueda ser usado para entrenamientos y validaciones. Su característica principal se basa en poder generar **K** particiones de la data, en las cuales cada una de estas particiones es utilizada para evaluar el modelo.
De esta forma se entrenan sobre las K-1 y se evalua sobre la partición restante, repitiendo este proceso hasta recorrer todas las particiones generadas, los resultados obtenidos son promediados generado un estimador con menos sesgo para las métricas de evaluación a considerar.  

<img src='./imgs/kfolds.png'>

https://towardsdatascience.com/cross-validation-k-fold-vs-monte-carlo-e54df2fc179


En el proyecto de predicción de sobrevivientes del Titanic se trabajó con un split que genero sets de entrenamiento, validación y pruebas. K-folds puede ser implementado si en lugar de generar el dataset de validación inicial, se trabaja con una función que permita particionar el dataset de entrenamiento K veces, de forma que se pueda generar predicciones sobre cada una de estas particiones y promediar los resultados. Siendo estos promedios los que guardaríasmo en la bitácora del proyecto y que usaríamos para seleccionar los mejores modelos de cada tipo. 

### Support Vector Machines (SVM)

Es un algoritmo de Machine Learning que permite resolver problemas de clasificación lineal y no lineal, basado en la aplicación de vectores de soporte que determinan no solo una frontera de decisión si no también un margen de seperación para la tarea a realizar.

<img src="./imgs/svm_graph.png">

https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47

#### Hipótesis

La hipótesis usada en SVM es una función por partes o condicional definida a partir de la la expresión lineal. 

<img src="./imgs/svm_hypothesis.png">

*Hands-On Machine Learning with Scikit-Learn & Tensorflow*

#### Función de Costo

La función de costos de SVM se conoce como bisagra(hinge).
Esta función busca encontrar el punto mínimo de la curva en ciertos valores de otra curva. Es decir busca el punto mínimo a partir de una restricción que se relaciona con la función. (Optimización con restricciones)


La funcion hinge no es un promedio de los errores si no más bien una suma de estos. En esta funcion el parámetro de regularización es denotado con la letra *C* y presenta un efecto igual al recíproco del término de regularización (1/lambda). 


<img src="./imgs/svm_costfunction.png">

*Hands-On Machine Learning with Scikit-Learn & Tensorflow*

#### Algoritmo de aprendizaje

#### Propiedades y comparación con otros algoritmos

SVM es un algoritmo que, a diferencia de otros algoritmos (e.g. regresión logística), no define una frontera de decisión. Esto debido a que SVM es un algoritmo que computa a su vez vectores de soporte que buscan ampliar la brecha que separa una clase a la otra, permitiendo la configuración de este margen generando clasificasiones de margen suave y duro. 

**Ventajas**
- Este algoritmo puede ser utilizado tanto para problemas de clasificasción como para problemas de regresión
- Maneja problemas lineales y no lineales 
- La posibilidad de implementar kernel trick reduce significativamente la complejidad del modelo

**Desventajas**
- Si la data no esta escalada el modelo se ve altamente afectado
- La aplicación de distintos kernel requiere del conocimiento de los distintos hiperparámetros (e.g. hiperparámetro C y Gamma) 

#### Kernel trick

Kernel es una funcion que mide la similitud entre dos vectores o funciones. Su aplicación permite generar un espacio latente, el cual es un espacio diferente obtenido a partir de ciertas transformaciones de los datos. 

El truco del kernel (kernel trick) es un concepto basado en *basis expansion* que permite replicar los efectos de transformar varias características sin tener que agregarlas realmente, evitando así el impacto computacional que genera el agregar una cierta cantidad de términos polinomiales. Esto con el objetivo de transformar las características en cuestión y generar un dataset separable para propósitos de la clasificación. 

En esencia el Kernel Trick se basa en ahorrar las transformacion de las características, de forma que el kernel es capaz de computar el producto punto de vectores transformados basado unicamente en los vectores originales, obviando así las transformaciones que serían necesarias de aplicar (phi). 

**Kernel trick**

<img src="./imgs/kernel_trick.png">

*https://medium.com/@sonalij663/what-is-kernel-tricks-bb176291e60f*

**Kernels**

<img src="./imgs/svm_kernel.png">

*Hands-On Machine Learning with Scikit-Learn & Tensorflow*